In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import json
from datetime import datetime, timedelta

In [3]:
base_url = 'https://www.indiaforums.com/show/chaskameter/'

In [4]:
data = []

for week in range (1,677):
    
    url = f"{base_url}{week}"
    
    page = requests.get(url)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    
    meta_tag = soup.find('meta', attrs={'name': 'description'})
    if meta_tag:
        date_string = meta_tag['content'].split('From ')[1].split(' to ')[0].strip()
    else:
        date_string = None
    
    rows = soup.find_all('tr')[1:]
    
    for row in rows:
        show_info = {}
        show_info['Start Date'] = date_string

        show_info['Rank'] = row.find('div', class_='order-number').text.strip()

        show_name_tag = row.find('div', class_='table-first-td-div-class').a
        show_info['Show Name'] = show_name_tag.text.strip()
        show_info['Show URL'] = 'https://www.indiaforums.com' + show_name_tag['href']

        show_info['Last Week'] = row.find('small').text.strip().replace('Last Week: ', '')

        metrics = row.find_all('li')
        #show_info['Buzz'] = metrics[0].text.strip().replace('Buzz :', '')
        show_info['Rating'] = metrics[1].text.strip().replace('Rating :', '')
        show_info['Views'] = metrics[2].text.strip().replace('Views :', '')
        show_info['Activity'] = metrics[3].text.strip().replace('Activity :', '')
        show_info['Followers'] = metrics[4].text.strip().replace('Followers :', '')
        #show_info['TRP'] = metrics[5].text.strip().replace('Trp :', '')
        show_info['Week_ID'] = week

        show_info['Total Points'] = row.find_all('td')[1].text.strip()

        data.append(show_info)

In [5]:
df = pd.DataFrame(data)

In [6]:
df.to_csv('chaskameter_show_ranking_all_weeks.csv', index=False)

In [45]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping of columns

In [46]:
df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
df['Start Date'] = df['Start Date'].dt.strftime('%Y-%m-%d')

In [47]:
client = pymongo.MongoClient("mongodb://localhost:27017")

In [48]:
data = df.to_dict(orient = "records")

In [49]:
db = client["TV_Shows"]

def insert_in_chunks(data, chunk_size=1000):
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        try:
            db.Shows.insert_many(chunk, ordered=False)
        except Exception as e:
            print(f"Error inserting chunk starting at index {i}: {e}")

insert_in_chunks(data)

print("Data insertion completed.")

Data insertion completed.


In [50]:
#print(db)

In [51]:
#db.Shows.insert_many(data)

In [52]:
df.head()

,Start Date,Rank,Show Name,Show URL,Last Week,Rating,Views,Activity,Followers,Week_ID,Total Points
0,2010-03-01,1,Miley Jab Hum Tum,https://www.indiaforums.com/show/miley-jab-hum...,0,294,137,115,88,1,729
1,2010-03-01,2,Dill Mill Gayye,https://www.indiaforums.com/show/dill-mill-gay...,0,172,275,175,100,1,722
2,2010-03-01,3,Mann Kee Awaaz Pratigya,https://www.indiaforums.com/show/mann-kee-awaa...,0,202,122,82,28,1,529
3,2010-03-01,4,Yeh Rishta Kya Kehlata Hai,https://www.indiaforums.com/show/yeh-rishta-ky...,1,209,143,44,32,1,523
4,2010-03-01,5,Sapna Babul Ka Bidaai,https://www.indiaforums.com/show/sapna-babul-k...,0,206,151,73,37,1,482
